# Representing numbers

You've already met two of Python's numerical types:

- `int` for integers, like 0 or 5235.
- `float` for floating point numbers like 3.1416 or 6.022e-23. 

The good news is that, most of the time, these are the only things you need to know about. The rest of this chapter is just a deeper dive into some concepts behind integers and floating point numbers. It's probably not a bad idea to skip it for now, and come back to it when you're more familiar with the basics.

Then again, if you just like numbers, read on!

## Complex numbers

There is another `type` for complex numbers like $(3 + 4\mathrm{i})$. It's called `complex`. To make a complex number in Python, we use `j` instead of i:

In [1]:
3 + 4j

(3+4j)

By the way, we can check the type of any object with the built-in `type()` function, which we can call just like the `print()` function:

In [2]:
type(3 + 4j)

complex

It's not common to need to use the `type` function in Python software, but it can be useful for debugging, especially while you're learning, because you can't usually tell anything about the type of an object from the name you give it.

When we start using the numerical library NumPy, we'll meet other kinds of numbers, such as unsigned integers and half-length floats. What are these strange creatures?

## A word about bits

There's a quirk about representing floating point numbers in computers. Because we'd like these numbers to have a well-defined size in memory, we limit them to a specific bit width — usually 64 bits. 

Maybe it's easier to understand one effect of bit width — the smallest and largest numbers you can represent — with integers. Clearly, with only 1 binary bit, only two numbers are possible: 0, and 1. With two bits, we can have 0, 1, 2, and 3. With 3 bits, we can have 0 to 7. With 8 bits, or a byte, we can represent integers from 0 up to 255. 

In particular, this is an *unsigned* integer. It results from a 'literal' interpretation of binary numbers, where the 1-byte word `11100011` represents 128 + 64 + 32 + 2 + 1 = 227<sub>10</sub>. 

Unsigned ints are fine for representing things like pixel values in a greyscale image. But sometimes we need negative numbers too — think of seismic amplitudes for instance. If we use an alternative representation, called two' complement, we can have signed integers. This scheme works by reserving all binary words starting with `1` as negative. This means we can't have any positive numbers greater than `01111111`, or 127. 

To find the two's complement representation of, say, `00011101` (29<sub>10</sub>), we first invert the digits, getting `11100010` and then add 1 to get `11100011`. This is the 8-bit two's complement representation of –29<sub>10</sub>. Notice that it's also the unsigned representation of 227<sub>10</sub> from the previous example.

## Floating point precision

When it comes to floats, the minimum and maximum expressible numbers aren't the only thing affected by the limited number of bits available. It also affects the precision of your arithmetic. 

Let's use the built-in function `format` to show the number 0.1 to 30 decimal places:

In [3]:
format(0.1, '.30f')

'0.100000000000000005551115123126'

As far as the computer is concerned, this *is* the number 0.1. Or, to put it another way, it can't tell the difference between this number and 0.1:

In [4]:
x = 0.1 - 0.100000000000000005551115123
format(x, '.30f')

'0.000000000000000000000000000000'

What's going on?

In a nutshell, computers represent floats a lot like scientists do when they use scientific notation. Imagine Avogadro's number, $6.022 \times 10^{23}$, or `+6.022e23` in code. This consists of a sign (`+`), a significand (`6.022`), a base (`10`), and an exponent (`23`).

The exact details vary a bit, but here's roughly how most systems represent 64-bit floating point numbers:

- 52 bits for the fractional component of the significand. This is always added to 1. (Analogously, using base 10 for simplicity, the number 5492 would thus become 1.5492.)
- 11 bits for the exponent, representing –1024 to +1023, to which the implicit base of 2 is raised.
- 1 bit to represent the sign of the number.

The upshot of all this is that sometimes you cannot rely on some ordinary mathematical truths:

In [5]:
0.1 + 0.1 + 0.1 == 0.3

False

We can get creative and check for this equality in other ways. For example, we can ensure the numbers differ by some tolerably small amount, usually called `epsilon`:

In [6]:
epsilon = 1e-12
0.1 + 0.1 + 0.1 - 0.3 < epsilon

True

By the way, we can use a similar trick to avoid dividing by zero when processing a large quantity of numbers — just add epsilon to the denominator.

What about the value of this epsilon, what should we choose? It can be any amount you consider insignificant, as long as it's larger than the 'machine epsilon', or macheps, which depends on the implementation details of the system you're using. Almost all Python implementations use the 64-bit IEEE 754 floating point standard. Python's `sys` library can give you information about its floats:

In [7]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

Later in this course we'll meet the numerical computing library, NumPy. In NumPy you can choose how many bytes are used to represent floating point numbers. It has its own version of `sys.float_info` called `np.finfo()`. Here's what it displays for half-precision, or 16-bit, floats:

In [8]:
import numpy as np

np.finfo(np.float64)

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

You can use the `resolution` value as epsilon, though note that even it suffers from its own curse:

In [9]:
np.finfo(np.float64).resolution

1e-15

To find out more about floating point numbers and precision, I recommend the following:

- More precise SEG-Y?, by Matt Hall. Agile Scientific blog, https://agilescientific.com/blog/2017/3/29/more-precise-segy
- [The perils of floating point](http://www.lahey.com/float.htm), by Bruce M. Bush.
- What every computer scientist should know about floating-point arithmetic, by David Goldberg. ACM Computing Surveys 23 (1), March 1991. [Available online.](http://perso.ens-lyon.fr/jean-michel.muller/goldberg.pdf)

## Decimal: another way to deal with floating points numbers

If you're in a situation where you really need to cope with floating point numbers, to be sure of the imprecision not biting you, there's a built-in library called `decimal`. 

In [10]:
0.1 + 0.2

0.30000000000000004

In [11]:
from decimal import Decimal

In [12]:
Decimal('0.1') + Decimal('0.2')

Decimal('0.3')

Notice that we pass strings. Why is that? Well, remember that Python *can't tell the difference* between 0.1 and, well, this:

In [13]:
Decimal(0.1)

Decimal('0.1000000000000000055511151231257827021181583404541015625')

...but it can interpret the string `'0.1'` as what we mean by `0.1`. So we use strings. Or we could also pass a tuple of sign, fractional digits in a tuple, and exponent, but it gets pretty tiresome:

In [14]:
Decimal((0,(1,), -1))

Decimal('0.1')

## Rational numbers

`decimal` has a sister library for representing fractions:

In [15]:
from fractions import Fraction

print(Fraction(1, 2))

1/2


In [16]:
Fraction(1, 2) * 3.141593

1.5707965

It can also interpret strings:

In [17]:
Fraction('7/22')

Fraction(7, 22)

And reduce fractions for you:

In [18]:
Fraction('45/545')

Fraction(9, 109)

Again, be careful with floating point precision:

In [19]:
Fraction(45/545)

Fraction(1487427399865485, 18014398509481984)

All pretty cool... but I've never used it.

## Other numeric types

There are some other numeric types in Python. For example, we can operate with binary, octal or hexadecimal numbers. Python regards all these numbers as `int`s, so we can use them anywhere we'd use an `int`:

In [20]:
0b11101001001101 + 0b110110000000001

42574

In [21]:
0o35115 + 0o66001

42574

In [22]:
0x3a4d + 0x6c01

42574

We can easily cast between different bases, though notice these result comes back as a string:

In [23]:
oct(0b110110000000001)

'0o66001'

In [24]:
hex(0o66001)

'0x6c01'

In [25]:
bin(0x6c01)

'0b110110000000001'

## You've got Python's number

The number one lesson from this chapter: computers are strange: numbers are not just numbers, but have multiple representations. The good news is that you will quickly get used to the idea of `int`s and `float`s, and most of the time you don't need to worry about the rest of it.